### 네이버 지식IN - 추석 선물

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm import tqdm

In [3]:
url= f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page=1'
driver = webdriver.Chrome()
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [4]:
lis = soup.select('.basic1 > li')
len(lis)

10

- 답변 갯수가 5개보다 큰 항목을 선택

In [5]:
li = lis[7]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=6&dirId=6130103&docId=454639453&qb=7LaU7ISdIOyEoOusvA==&enc=utf8&section=kin&rank=8&search_sort=0&spq=0'

In [6]:
driver.get(sub_url)
time.sleep(2)

In [7]:
# 더보기 클릭 회수 구하기
answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
click_count = (int(answer_num) - 1) // 5
answer_num, click_count

('8', 1)

In [8]:
# 더 보기 횟수 만큼 클릭하기
for _ in range(click_count):
    driver.find_element(By.ID, 'nextPageButton').click()
    time.sleep(2)

In [9]:
# 펼처진 화면을 BeautifulSoup으로 파싱
sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
divs = sub_soup.select('.answer-content__list._answerList > div')
len(divs)

8

In [10]:
# 답변 목록에서 개별 답변을 선택하고 글을 추출하기
div = divs[0]
ps = div.select('p.se-text-paragraph.se-text-paragraph-align-')
len(ps)

45

In [11]:
answer = ''
for p in ps:
    text = p.get_text().strip()
    if text == '\u200b':
        continue
    answer += text + '\n'
answer[:100]

'많은 인원수 준비 하실려면 부담이 크시겠어요\n연령대도 다양하시고\n남녀노소 가리지 않고 해드릴수 있는 선물을 추천 해드릴려고 합니다\n곧 풍성한 추석이 몇일 남지 않았어요\n\n부모님추석'

In [12]:
# 답변 목록에 있는 모든 답변을 추출하기
answer_list = []
for div in divs:
    ps = div.select('p.se-text-paragraph.se-text-paragraph-align-')
    answer = ''
    for p in ps:
        text = p.get_text().strip()
        if text == '\u200b' or text == '':
            continue
        answer += text + '\n'
    answer_list.append(answer)

In [13]:
len(answer_list)

8

- 한 화면에 있는 10개의 글에서 모든 답변을 추출하기

In [14]:
answer_list = []
for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)
    time.sleep(2)
    
    answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
    click_count = (int(answer_num) - 1) // 5
    
    for _ in range(click_count):
        driver.find_element(By.ID, 'nextPageButton').click()
        time.sleep(2)
        
        sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = sub_soup.select('.answer-content__list._answerList > div')
    
        for div in divs:
            ps = div.select('p.se-text-paragraph.se-text-paragraph-align-')
            answer = ''
            for p in ps:
                text = p.get_text().strip()
                if text == '\u200b' or text == '':
                    continue
                answer += text + '\n'
            answer_list.append(answer)


KeyboardInterrupt: 

In [ ]:
len(answer_list)

57

- 1~10페이지에 대한 답변 가져오기

In [ ]:
answer_list = []
driver = webdriver.Chrome()
for page in tqdm(range(1, 11)):
    url= f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page={page}'
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)
        
        answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
        click_count = (int(answer_num) - 1) // 5
        
        for _ in range(click_count):
            driver.find_element(By.ID, 'nextPageButton').click()
            time.sleep(2)
        
        sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = sub_soup.select('.answer-content__list._answerList > div')
    
        for div in divs:
            ps = div.select('p.se-text-paragraph.se-text-paragraph-align-')
            answer = ''
            for p in ps:
                text = p.get_text().strip()
                if text == '\u200b' or text == '':
                    continue
                answer += text + '\n'
            answer_list.append(answer)

        driver.back()
        time.sleep(2)

driver.close()

In [15]:
import threading

answer_list = []
driver1 = webdriver.Chrome()
driver2 = webdriver.Chrome()
driver3 = webdriver.Chrome()
driver4 = webdriver.Chrome()
driver5 = webdriver.Chrome()

def answer_func(driver, url):
    driver.get(url)
    time.sleep(2)
    
    answer_num = driver.find_element(By.CSS_SELECTOR, '._answerCount.num').text
    click_count = (int(answer_num) - 1) // 5
    
    for _ in range(click_count):
        driver.find_element(By.ID, 'nextPageButton').click()
        time.sleep(2)
        
    sub_soup = BeautifulSoup(driver.page_source, 'html.parser')
    divs = sub_soup.select('.answer-content__list._answerList > div')
    
    for div in divs:
        ps = div.select('p.se-text-paragraph.se-text-paragraph-align-')
        answer = ''
        for p in ps:
            text = p.get_text().strip()
            if text == '\u200b' or text == '':
                continue
            answer += text + '\n'
        answer_list.append(answer)

threads = []
for page in tqdm(range(1, 11)):
    url= f'https://kin.naver.com/search/list.naver?query={quote("추석 선물")}&page={page}'
    driver1.get(url)
    soup = BeautifulSoup(driver1.page_source, 'html.parser')
    lis = soup.select('.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        t = threading.Thread(target=answer_func, args=(driver2, sub_url))
        threads.append(t)
        t.start()

for t in threads:
    t.join()

driver1.close()
driver2.close()
driver3.close()
driver4.close()
driver5.close()

len(answer_list)



    



    

100%|██████████| 10/10 [00:03<00:00,  2.62it/s]
Exception in thread Thread-55 (answer_func):
Traceback (most recent call last):
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 714, in urlopen
Exception in thread Thread-76 (answer_func):
Traceback (most recent call last):
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 714, in urlopen
Exception in thread Thread-22 (answer_func):
Traceback (most recent call last):
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 714, in urlopen
Exception in thread Thread-104 (answer_func):
Traceback (most recent call last):
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 714, in urlopen
Exception in thread Thread-54 (answer_func):
Traceback (most recent call last):
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 714, in urlopen
Exception in thread Thread-59 (answer_func):
Traceback (most recent cal

KeyboardInterrupt: 

Exception in thread Thread-79 (answer_func):
Traceback (most recent call last):
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 714, in urlopen
    httplib_response = self._make_request(
                       ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\h\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 415, in _make_request
    conn.request(method, url, **http